In [1]:
import torch
import pandas as pd
import time
import re

In [2]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
dev

device(type='cuda')

In [3]:
oneg = pd.read_csv('../onegin.txt', sep='\n', header=None)

In [4]:
# читаем стихи
max_len = 0
phrases = []
for ind in range(len(oneg)):
    line = oneg[0][ind]
    if re.search(r'\t', line):
        # удалить ^\t\t
        line = re.sub(r'\t\t', '', line)
        # удалить …………
        line = re.sub(r'…', '', line)
        # удалить все после [d]
        line = re.sub(r'\[\d*\].*$', '', line)
        # удалить все в квадратных скобках
        line = re.sub(r'\[.*\]', '', line)
        # удалить все анлгийские буквы
        line = re.sub(r'[abcdefghijklmnopqrstuvwxyz]', '', line)
        # удалить \xa0
        line = re.sub(r'\xa0', ' ', line)
              
        if len(line) > 0:
            phrases.append(line)
            line_len = len(line)
            if line_len > max_len:
                max_len = line_len

In [5]:
max_len

37

In [6]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [7]:
CHARS = set(' абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [8]:
MAX_LEN = max_len+1

In [9]:
X = torch.zeros((len(text), MAX_LEN), dtype=int)
X = X.to(dev)

In [10]:
X.shape

torch.Size([1453, 38])

In [11]:
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [12]:
X[0:1]

tensor([[ 0, 33, 22, 19, 31,  5, 25, 33, 30, 32,  6, 16, 19, 14, 33, 31, 18,  1,
         17, 33,  9, 13,  3, 13, 18, 28, 17,  7,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]], device='cuda:0')

In [13]:
class Network(torch.nn.Module):

    def __init__(self, dev):
        super(Network, self).__init__()
        self.dev = dev
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 50).to(self.dev)
        self.gru = torch.nn.RNN(50, 128, num_layers = 3, nonlinearity = 'relu', batch_first=True).to(self.dev)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR)).to(self.dev)

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [14]:
class NetworkGRU(torch.nn.Module):

    def __init__(self, dev):
        super(NetworkGRU, self).__init__()
        self.dev = dev
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 50).to(self.dev)
        self.gru = torch.nn.GRU(50, 128, num_layers = 3, batch_first=True).to(self.dev)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR)).to(self.dev)

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [15]:
class NetworkLSTM(torch.nn.Module):

    def __init__(self, dev):
        super(NetworkLSTM, self).__init__()
        self.dev = dev
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 50).to(self.dev)
        self.gru = torch.nn.LSTM(50, 128, num_layers = 3, batch_first=True).to(self.dev)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR)).to(self.dev)

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [16]:
model = Network(dev)
model.forward(X[0:1])[0].shape

torch.Size([1, 38, 35])

In [17]:
modelGRU = NetworkGRU(dev)
modelGRU.forward(X[0:1])[0].shape

torch.Size([1, 38, 35])

In [18]:
modelLSTM = NetworkLSTM(dev)
modelLSTM.forward(X[0:1])[0].shape

torch.Size([1, 38, 35])

In [44]:
def generate_sentence(line):
    sentence = [w for w in line]
    #sentence = ['к', 'а', 'к', ' ', 'р', 'а', 'н', 'о',' ']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        X = X.to(dev)
        if i == 0:
            result, state = model.forward(X)
        else:
            result, state = model.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
        
    line = ''.join(sentence)
    line = re.sub(r'none.*$', '', line)
    print(line)

In [45]:
def generate_sentence_GRU(line):
    sentence = [w for w in line]
    #sentence = ['к', 'а', 'к', ' ', 'р', 'а', 'н', 'о',' ']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        X = X.to(dev)
        if i == 0:
            result, state = modelGRU.forward(X)
        else:
            result, state = modelGRU.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])

    line = ''.join(sentence)
    line = re.sub(r'none.*$', '', line)
    print(line)        

In [46]:
def generate_sentence_LSTM(line):
    sentence = [w for w in line]
    #sentence = ['к', 'а', 'к', ' ', 'р', 'а', 'н', 'о',' ']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        X = X.to(dev)
        if i == 0:
            result, state = modelLSTM.forward(X)
        else:
            result, state = modelLSTM.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
        
    line = ''.join(sentence)
    line = re.sub(r'none.*$', '', line)
    print(line)

In [47]:
generate_sentence('как рано')

как раноалумонч  ьн


In [48]:
generate_sentence_GRU('как рано')

как раноос 


In [49]:
generate_sentence_LSTM('как рано')

как раноо  саси  ссттт ввееаааиоен 


In [25]:
lrate = .1

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lrate)

criterionGRU = torch.nn.CrossEntropyLoss()
optimizerGRU = torch.optim.SGD(modelGRU.parameters(), lr=lrate)

criterionLSTM = torch.nn.CrossEntropyLoss()
optimizerLSTM = torch.optim.SGD(modelLSTM.parameters(), lr=lrate)

In [26]:
count = 0

for ep in range(1000):
    
    # RNN section
    start = time.time()
    train_loss = 0.
    train_passed = 0
    
    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1
    end = time.time()
        
    # GRU section
    startGRU = time.time()
    train_lossGRU = 0.
    train_passedGRU = 0
    
    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizerGRU.zero_grad()
        answers, _ = modelGRU.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterionGRU(answers, Y_batch)
        train_lossGRU += loss.item()

        loss.backward()
        optimizerGRU.step()
        train_passedGRU += 1
    endGRU = time.time()
    
    # LSTM section
    startLSTM = time.time()
    train_lossLSTM = 0.
    train_passedLSTM = 0
    
    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizerLSTM.zero_grad()
        answers, _ = modelLSTM.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterionLSTM(answers, Y_batch)
        train_lossLSTM += loss.item()

        loss.backward()
        optimizerLSTM.step()
        train_passedLSTM += 1
    endLSTM = time.time()
    
    count += 1
    if count == 20:
        count = 0
        trloss = train_loss / train_passed
        trlossGRU = train_lossGRU / train_passedGRU
        trlossLSTM = train_lossLSTM / train_passedLSTM
        print("Epoch {}. Time,GRU,LSTM: {:.3f},{:.3f},{:.3f}, Train loss,GRU,LSTM: {:.3f},{:.3f},{:.3f}, delta RGU,LSTM:{:.3f},{:.3f}".format(ep, end - start, endGRU - startGRU, endLSTM - startLSTM, trloss, trlossGRU,trlossLSTM, trloss - trlossGRU,trloss - trlossLSTM))
        generate_sentence('как рано ')
        generate_sentence_GRU('как рано ')        
        generate_sentence_LSTM('как рано ')        

Epoch 19. Time,GRU,LSTM: 0.165,0.190,0.205, Train loss,GRU,LSTM: 1.901,2.052,2.174, delta RGU,LSTM:-0.151,-0.273
как рано а асал  с сет  соосер соол    сел  соо
как рано    оооо ооооооо о   nonenonenonenone  ооооnonenonenonenone ооо
как рано   none                                   
Epoch 39. Time,GRU,LSTM: 0.170,0.190,0.200, Train loss,GRU,LSTM: 1.725,2.004,2.070, delta RGU,LSTM:-0.280,-0.345
как рано а  пала с прора стсооса стое    пто  п
как рано о  оооо ооооое noneонnonenonenonenonenone nonenonenonenonenonenonenonenonenonenonenonenonenonenone
как рано            оооооооооо    nonenonenonenonenone      nonenone
Epoch 59. Time,GRU,LSTM: 0.165,0.190,0.205, Train loss,GRU,LSTM: 1.647,1.969,2.031, delta RGU,LSTM:-0.321,-0.384
как рано а  сас  с пртлтвстррееааае аа   nonenonenoneс none
как рано о  оооо ооооо nonenoneнnonenonenonenonenoneнnonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
как рано          ооооооооооо  nonenonenonenonenonenonenonenone  nonenonenonenonenonenone
E

In [27]:
def generate_sentence2(line):
    generate_sentence(line)
    generate_sentence_GRU(line)
    generate_sentence_LSTM(line)

In [50]:
generate_sentence2('как рано')

как раноалумонч  ьн
как раноос 
как раноо  саси  ссттт ввееаааиоен 


In [51]:
generate_sentence2('привет чувак')

привет чувакеа есесагс ирлсмняе ояи о
привет чувакооеыт навсл вн 
привет чувакоолольсесо    ьнаян


In [52]:
generate_sentence2(' ')

 нам от разне не всегда по моды
 соседа своей 
 от полна столине стола


In [53]:
generate_sentence2('а')

ана призлостя
атель не в поставил свет
а столи в серенный сердень


In [54]:
generate_sentence2('схо')

схоеодйро евог лоа тм
схоеодйд наие  тв
схооотт ьои м посолттаиа


In [55]:
generate_sentence2('карета')

каретаалуйи
каретаосе   мьлв
каретао и    с ссссепееее ннт


In [56]:
generate_sentence2('з')

зах он
зой поставил своей
зоре полителья сердень


In [58]:
generate_sentence2('так думал молодой ')

так думал молодой ам
так думал молодой о  
так думал молодой ол не  л иолиньй пнона


In [59]:
generate_sentence2('с новым годом ')

с новым годом евежый ортнв
с новым годом епавый иорав пгелл
с новым годом ооамо  иолел он  
